In [ ]:
# Install the OpenAI Python package.
%pip install openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [ ]:
# Code example of OpenAI communication

from openai import OpenAI

client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='a0BIj0000026Ph1MAE'
)

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [ ]:
# Code example of reading input data

import pandas as pd
from IPython.display import display

def read_data_frame(document_id, sheet_name):
    export_link = f"https://docs.google.com/spreadsheets/d/{document_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
    return  pd.read_csv(export_link)

document_id = '14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
products_df = read_data_frame(document_id, 'products')
emails_df = read_data_frame(document_id, 'emails')

# Display first 3 rows of each DataFrame
display(products_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


In [ ]:
from google.colab import auth
from google.auth import default
import gspread

# Authenticate user
auth.authenticate_user()

# Create Google Sheets client
creds, _ = default()
gc = gspread.authorize(creds)


In [ ]:

# This code goes after creating google client
output_document = gc.create('Solving Business Problems with AI - Output')

# Create 'email-classification' sheet
email_classification_sheet = output_document.add_worksheet(title="email-classification", rows=50, cols=2)
email_classification_sheet.update([['email ID', 'category']], 'A1:B1')
set_with_dataframe(email_classification_sheet, email_classification_df)


# Create 'order-status' sheet
order_status_sheet = output_document.add_worksheet(title="order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')
set_with_dataframe(order_status_sheet, order_status_df)

# Create 'order-response' sheet
order_response_sheet = output_document.add_worksheet(title="order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')
set_with_dataframe(order_response_sheet,order_response_df )

# Create 'inquiry-response' sheet
inquiry_response_sheet = output_document.add_worksheet(title="inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')
set_with_dataframe(inquiry_response_sheet,inquiry_response_df )

# Share the spreadsheet publicly
output_document.share('', perm_type='anyone', role='reader')


# Print the shareable link
print(f"Shareable link: https://docs.google.com/spreadsheets/d/{output_document.id}")


Shareable link: https://docs.google.com/spreadsheets/d/1j6Ca-TIL76CNqeD3tUhQFroj1FLwZwXav5wHJiAPnE0


# Task 1. Classify emails

In [ ]:
import time

def classify_email(email_content):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": f"Classify the following email as either 'order request' or 'product inquiry' . Write either 'order request' or 'product inquiry only for' : {email_content}"}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error classifying email: {e}")
        return "error"  # Default value in case of failure
    finally:
        time.sleep(0.5)  # Pause to avoid rate-limiting
# Apply classification to emails DataFrame

emails_df['category'] =  emails_df['message'].apply(classify_email)

# Prepare DataFrame for upload
email_classification_df = emails_df[['email_id', 'category']]

In [ ]:
email_classification_df

,email_id,category
0,E001,order request
1,E002,order request
2,E003,product inquiry
3,E004,order request
4,E005,product inquiry
5,E006,product inquiry
6,E007,order request
7,E008,order request
8,E009,product inquiry
9,E010,order request



# Task 2. Process order requests

In [ ]:
import pandas as pd
import re
from IPython.display import display

def extract_product_ids(message):

    # Match product IDs with pattern of uppercase letters followed by numbers
    product_ids = re.findall(r'[A-Z]{2,}[0-9]+', message)
    return product_ids

# Apply the function to extract product IDs
emails_df['product_ids'] = emails_df['message'].apply(extract_product_ids)

# Display the updated DataFrame
display(emails_df)

,email_id,subject,message,category,product_ids
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request,[LTH0976]
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",order request,[VBT2345]
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,[LTH1098]
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In...",order request,[SFT1098]
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,[CSH1098]
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,[CBT8901]
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,"[CLF2109, FZZ1098]"
7,E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your Versatile...",order request,[]
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,[DHN0987]
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901...",order request,[RSG8901]


In [ ]:
nemails_df=emails_df.copy()
display(nemails_df)

,email_id,subject,message,category,product_ids
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request,[LTH0976]
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",order request,[VBT2345]
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,[LTH1098]
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In...",order request,[SFT1098]
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,[CSH1098]
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,[CBT8901]
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,"[CLF2109, FZZ1098]"
7,E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your Versatile...",order request,[]
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,[DHN0987]
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901...",order request,[RSG8901]


In [ ]:

# Function to retrieve stock values for product IDs
def get_stocks_for_products(product_ids):
    if not product_ids:
        return None
    stock = [
        products_df.loc[products_df['product_id'] == pid, 'stock'].values[0]
        if pid in products_df['product_id'].values else None
        for pid in product_ids
    ]
    return stock

# Add a stocks column to emails_df based on product_ids
nemails_df['stock'] = nemails_df['product_ids'].apply(get_stocks_for_products)

display(nemails_df)

,email_id,subject,message,category,product_ids,stock
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request,[LTH0976],[4]
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",order request,[VBT2345],[4]
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,[LTH1098],[7]
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In...",order request,[SFT1098],[8]
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,[CSH1098],[3]
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,[CBT8901],[2]
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,"[CLF2109, FZZ1098]","[2, 2]"
7,E008,Ordering a Versatile Scarf-like item,"Hello, I'd want to order one of your Versatile...",order request,[],None
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,[DHN0987],[None]
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901...",order request,[RSG8901],[1]


In [ ]:

# Expand rows with multiple product_ids into separate rows
expanded_rows = []
for _, row in nemails_df.iterrows():
    product_ids = row["product_ids"] if row["product_ids"] is not None else []
    stocks = row["stock"] if row["stock"] is not None else []
    if len(product_ids) == len(stocks):  # Ensure matching lengths
        for pid, stock in zip(product_ids, stocks):
            new_row = row.copy()
            new_row["product_ids"] = pid
            new_row["stock"] = stock
            expanded_rows.append(new_row)
    else:
        print(f"Row mismatch or empty: {row}")


# Reset index for the new DataFrame

# Create the expanded DataFrame
expanded_df = pd.DataFrame(expanded_rows)

# Reset index for the new DataFrame
expanded_df.reset_index(drop=True, inplace=True)
expanded_df

,email_id,subject,message,category,product_ids,stock
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request,LTH0976,4.0
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",order request,VBT2345,4.0
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,LTH1098,7.0
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In...",order request,SFT1098,8.0
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,CSH1098,3.0
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,CBT8901,2.0
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,CLF2109,2.0
7,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,FZZ1098,2.0
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,DHN0987,NaN
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901...",order request,RSG8901,1.0


In [ ]:

def extract_quantity_with_gpt(message,stock,id):
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": f"Given the product ID: {id} and the total stock: {stock}, extract the number of items requested for this product from the following message: '{message}'. If the quantity is not explicitly mentioned in the message, assume it to be 1. Only output a single integer. Example output: 5"}
            ]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error extracting quantity: {e}")
        return "error"  # Default value in case of failure
    finally:
        time.sleep(0.5)  # Pause to avoid rate-limiting

# Apply GPT function to emails_df
expanded_df["quantities"] = expanded_df.apply(
    lambda row: [extract_quantity_with_gpt(row["message"],row["stock"],row["product_ids"])]  if row["category"] == "order request" else [],
    axis=1
)

# Display the updated DataFrame
display(expanded_df)

,email_id,subject,message,category,product_ids,stock,quantities
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request,LTH0976,4.0,[4]
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",order request,VBT2345,4.0,[1]
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,LTH1098,7.0,[]
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In...",order request,SFT1098,8.0,[3]
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,CSH1098,3.0,[]
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,CBT8901,2.0,[]
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,CLF2109,2.0,[5]
7,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,FZZ1098,2.0,[2]
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,DHN0987,NaN,[]
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901...",order request,RSG8901,1.0,[1]


In [ ]:
expanded_df["quantities"] = expanded_df["quantities"].apply(
    lambda x: float(x) if isinstance(x, (int, float)) else (float(x[0]) if isinstance(x, list) and x else 0)
)
order_df=expanded_df.copy()
display(order_df)
order_df.info()

,email_id,subject,message,category,product_ids,stock,quantities
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT...",order request,LTH0976,4.0,4.0
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V...",order request,VBT2345,4.0,1.0
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,LTH1098,7.0,NaN
3,E004,Buy Infinity Scarves Order,"Hi, I'd like to order three to four SFT1098 In...",order request,SFT1098,8.0,3.0
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,CSH1098,3.0,NaN
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,CBT8901,2.0,NaN
6,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,CLF2109,2.0,5.0
7,E007,"Order for Beanies, Slippers","Hi, this is Liz. Please send me 5 CLF2109 Cabl...",order request,FZZ1098,2.0,2.0
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,DHN0987,NaN,NaN
9,E010,Purchase Retro Sunglasses,"Hello, I would like to order 1 pair of RSG8901...",order request,RSG8901,1.0,1.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   email_id     18 non-null     object 
 1   subject      12 non-null     object 
 2   message      18 non-null     object 
 3   category     18 non-null     object 
 4   product_ids  18 non-null     object 
 5   stock        17 non-null     float64
 6   quantities   9 non-null      float64
dtypes: float64(2), object(5)
memory usage: 1.1+ KB


In [ ]:

orders_df_filtered = order_df[order_df["category"] == "order request"]
orders_df_filtered = pd.DataFrame(orders_df_filtered)


# Process orders and create the order-status sheet
order_status_records = []

for _, row in orders_df_filtered.iterrows():
    email_id = row["email_id"]
    product_id = row["product_ids"]
    stock = row["stock"]
    quantity = row["quantities"]

    if stock >= quantity:
        # Order can be fulfilled
        order_status_records.append({
            "email_id": email_id,
            "product_id": product_id,
            "quantity": quantity,
            "status": "created"
        })
        # Update stock
        stock -= quantity
    else:
        # Order cannot be fulfilled due to insufficient stock
        order_status_records.append({
            "email_id": email_id,
            "product_id": product_id,
            "quantity": quantity,
            "status": "out of stock"
        })

# Create the order-status DataFrame
order_status_df = pd.DataFrame(order_status_records)

order_status_df

,email_id,product_id,quantity,status
0,E001,LTH0976,4.0,created
1,E002,VBT2345,1.0,created
2,E004,SFT1098,3.0,created
3,E007,CLF2109,5.0,out of stock
4,E007,FZZ1098,2.0,created
5,E010,RSG8901,1.0,created
6,E018,RSG8901,2.0,out of stock
7,E019,FZZ1098,1.0,created
8,E023,CGN2345,5.0,out of stock


In [ ]:
# Generate response emails based on the order-status results
responses = []

for _, row in order_status_df.iterrows():
    email_id = row["email_id"]
    product_id = row["product_id"]
    quantity = row["quantity"]
    status = row["status"]

    if status == "created":
        response = (
            f"Dear Customer,\n\n"
            f"Thank you for your order. We are pleased to inform you that your order for "
            f"{int(quantity)} units of product ID {product_id} has been successfully processed. "
            f"The items will be prepared for shipping shortly.\n\n"
            f"Thank you for choosing our services.\n\n"
            f"Best regards,\nCustomer Service Team"
        )
    elif status == "out of stock":
        response = (
            f"Dear Customer,\n\n"
            f"Thank you for your order. Unfortunately, we are unable to fulfill your order for "
            f"{int(quantity)} units of product ID {product_id} due to insufficient stock. "
            f"We sincerely apologize for the inconvenience.\n\n"
            f"As an alternative, you may choose to wait for a restock or explore other similar products "
            f"available on our platform.\n\n"
            f"If you have any further questions, feel free to reach out to us.\n\n"
            f"Best regards,\nCustomer Service Team"
        )

    responses.append({"email_id": email_id, "response": response})

# Create the order-response DataFrame
order_response_df = pd.DataFrame(responses)

print(order_response_df)

  email_id                                           response
0     E001  Dear Customer,\n\nThank you for your order. We...
1     E002  Dear Customer,\n\nThank you for your order. We...
2     E004  Dear Customer,\n\nThank you for your order. We...
3     E007  Dear Customer,\n\nThank you for your order. Un...
4     E007  Dear Customer,\n\nThank you for your order. We...
5     E010  Dear Customer,\n\nThank you for your order. We...
6     E018  Dear Customer,\n\nThank you for your order. Un...
7     E019  Dear Customer,\n\nThank you for your order. We...
8     E023  Dear Customer,\n\nThank you for your order. Un...


# Task 3. Handle product inquiry

In [ ]:
inquiry_emails = expanded_df[expanded_df["category"] == "product inquiry"]
inquiry_emails

,email_id,subject,message,category,product_ids,stock,quantities
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,LTH1098,7.0,NaN
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,CSH1098,3.0,NaN
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,CBT8901,2.0,NaN
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,DHN0987,NaN,NaN
10,E011,Question on Retro Sunglasses Description,"Hi there, The description for the RSG8901 Retr...",product inquiry,RSG8901,1.0,NaN
13,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,SDE2345,1.0,NaN
14,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,DJN8901,5.0,NaN
15,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,RGD7654,5.0,NaN
16,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,CRD3210,5.0,NaN


In [ ]:
products_df.set_index("product_id", inplace=True)

KeyError: "None of ['product_id'] are in the columns"

In [ ]:
# Function to retrieve product details
def get_product_details(product_id):
    try:
        # Fetch product details efficiently
        details = products_df.loc[product_id]
        return {
            "name": details["name"],
            "category": details["category"],
            "description": details["description"],
            "price": details["price"],
            "stock": details["stock"],
        }
    except KeyError:
        return None  # Handle case where product_id is not found


inquiry_emails = inquiry_emails.copy()

# Add product details for each inquiry
def attach_product_details(product_id):
    return get_product_details(product_id)

# Add the details column to the inquiry_emails DataFrame
inquiry_emails["product_details"] = inquiry_emails["product_ids"].apply(attach_product_details)
inquiry_emails

,email_id,subject,message,category,product_ids,stock,quantities,product_details
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and...",product inquiry,LTH1098,7.0,NaN,"{'name': 'Leather Backpack', 'category': 'Bags..."
4,E005,Inquiry on Cozy Shawl Details,"Good day, For the CSH1098 Cozy Shawl, the desc...",product inquiry,CSH1098,3.0,NaN,"{'name': 'Cozy Shawl', 'category': 'Accessorie..."
5,E006,NaN,"Hey there, I was thinking of ordering a pair o...",product inquiry,CBT8901,2.0,NaN,"{'name': 'Chelsea Boots', 'category': 'Men's S..."
8,E009,Pregunta Sobre Gorro de Punto Grueso,"Hola, tengo una pregunta sobre el DHN0987 Gorr...",product inquiry,DHN0987,NaN,NaN,None
10,E011,Question on Retro Sunglasses Description,"Hi there, The description for the RSG8901 Retr...",product inquiry,RSG8901,1.0,NaN,"{'name': 'Retro Sunglasses', 'category': 'Acce..."
13,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,SDE2345,1.0,NaN,"{'name': 'Saddle Bag', 'category': 'Bags', 'de..."
14,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,DJN8901,5.0,NaN,"{'name': 'Distressed Jeans', 'category': 'Men'..."
15,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,RGD7654,5.0,NaN,"{'name': 'Rugged Denim Jacket', 'category': 'M..."
16,E021,NaN,So I've bought quite large collection of vinta...,product inquiry,CRD3210,5.0,NaN,"{'name': 'Corduroy Pants', 'category': 'Women'..."


In [ ]:
import time

# Function to generate response using ChatGPT
def generate_inquiry_response(email_id, product_details, message):
    try:
        if product_details:
            # Structure the response for product details
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": (
                            f"Here are the product details:"
                            f"Name: {product_details['name']}"
                            f"Category: {product_details['category']}"
                            f"Description: {product_details['description']}"
                            f"Price: ${product_details['price']}\n"
                            f"Stock: {product_details['stock']} available."
                            f"Now respond to a customer inquiry professionally, thanking them for the inquiry, "
                            f"and sharing these details in a helpful tone."
                            f"Format : Dear Customer,......  Best regards,\n Customer Service Team"
                            f"DONT PUT STARS"
                        )
                    }
                ]
            )
        else:
            # Response for missing product details
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {
                        "role": "user",
                        "content": (
                            f"The product mentioned in this inquiry is not found in the catalog. "
                            f"Write a polite response to the customer explaining that the product is unavailable, "
                            f"and ask for more details if needed."
                            f"Format : Dear Customer,......  Best regards,\n Customer Service Team"
                            f"DONT PUT STARS"
                        )
                    }
                ]
            )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error generating response for email ID {email_id}: {e}")
        return "Error generating response."
    finally:
        time.sleep(0.5)  # Pause to prevent hitting API rate limits

# Generate responses for product inquiries
inquiry_responses = []

for _, row in inquiry_emails.iterrows():
    email_id = row["email_id"]
    product_details = row["product_details"]
    message = row["message"]

    # Generate response using ChatGPT
    response = generate_inquiry_response(email_id, product_details, message)
    inquiry_responses.append({"email_id": email_id, "response": response})

# Create a DataFrame for inquiry responses
inquiry_response_df = pd.DataFrame(inquiry_responses)

inquiry_response_df

,email_id,response
0,E003,"Dear Customer,\n\nThank you for reaching out t..."
1,E005,"Dear Customer,\n\nThank you for reaching out t..."
2,E006,"Dear Customer,\n\nThank you for reaching out t..."
3,E009,"Dear Customer,\n\nThank you for reaching out t..."
4,E011,"Dear Customer,\n\nThank you for reaching out t..."
5,E021,"Dear Customer,\n\nThank you for your inquiry a..."
6,E021,"Dear Customer,\n\nThank you for reaching out t..."
7,E021,"Dear Customer,\n\nThank you for your inquiry a..."
8,E021,"Dear Customer,\n\nThank you for reaching out t..."
